## Unsloth: Optimizing Training and Inference Performance

For many software algorithms, the performance does not only depend on the number and kind of calculations performed. Instead, the exact order and the size of chunks has an enormous influence on the calculation speed.
For large language models, a library called `unsloth` contains optimized GPU kernels created by manually deriving all compute heavy math steps. By using these optimized kernels, a significant speed-up can be obtained.

### Key Techniques in Unsloth:

1. **Efficient Data Loading**: Optimizing data pipelines to reduce latency and improve throughput during training.
2. **Batching and Padding Strategies**: Dynamically adjusting batch sizes and minimizing padding to optimize memory usage.
3. **Half-Precision and Quantized Inference**: Using mixed precision or quantized models to speed up inference and reduce memory footprint.
4. **Model Pruning and Distillation**: Reducing the size of the model by removing redundant parameters or training smaller models to mimic larger ones.

### Benefits of Unsloth:

- **Reduced Training Time**: Optimizing data loading and model architecture reduces the time required for each epoch.
- **Lower Memory Usage**: Using techniques like mixed precision and quantization reduces the amount of GPU memory required.
- **Faster Inference**: Optimizing the model for deployment can significantly reduce latency during inference.

### Hands-On Example: Efficient Data Loading and Mixed Precision Training

In this example, we take the example from the previous notebook ("PEFT") and adjust them to use `unsloth`.

In [1]:
# Import libraries
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig, pipeline, TrainingArguments
from trl import SFTTrainer, SFTConfig
## Instead of:
# from transformers import AutoModelForCausalLM, AutoTokenizer
## use:
from unsloth import FastLanguageModel

/tmp/ipykernel_443176/2490468213.py:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothKTOTrainer: No module named 'UnslothKTOTrainer'. Using tempfile instead!


In [2]:
# Choose a model and load tokenizer and model (using 4bit quantization):
model_name = "unsloth/Phi-3.5-mini-instruct-bnb-4bit"

## Instead of:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(...)
## use: 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    cache_dir='/nvme/scratch/edu28/models',
    ## Instead of:
    # quantization_config=BitsAndBytesConfig(...)
    ## use:
    load_in_4bit=True,
    # device_map='cuda:0',
    trust_remote_code=True
)
tokenizer.padding_side = 'right'
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0


In [3]:
# Load the guanaco dataset
guanaco_train = load_dataset("timdettmers/openassistant-guanaco", cache_dir='/nvme/scratch/edu28/data', split='train')
guanaco_test = load_dataset("timdettmers/openassistant-guanaco", cache_dir='/nvme/scratch/edu28/data', split='test')

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


In [4]:
guanaco_train = guanaco_train.map(lambda entry: {
    'question1': entry['text'].split('###')[1].removeprefix(' Human: '),
    'answer1': entry['text'].split('###')[2].removeprefix(' Assistant: ')
})
guanaco_train = guanaco_train.map(lambda entry: {'messages': [
    {'role': 'user', 'content': entry['question1']},
    {'role': 'assistant', 'content': entry['answer1']}
]})

In [5]:
guanaco_test = guanaco_test.map(lambda entry: {
    'question1': entry['text'].split('###')[1].removeprefix(' Human: '),
    'answer1': entry['text'].split('###')[2].removeprefix(' Assistant: ')
})

guanaco_test = guanaco_test.map(lambda entry: {'messages': [
    {'role': 'user', 'content': entry['question1']},
    {'role': 'assistant', 'content': entry['answer1']}
]})

In [6]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [7]:
%%time
# We now have a look at the prediction before finetuning:
output_before_training = pipe(guanaco_test[3]['messages'][:1], max_new_tokens=256, return_full_text=False, do_sample=False)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


CPU times: user 8.24 s, sys: 265 ms, total: 8.51 s
Wall time: 8.74 s


In [8]:
print(output_before_training[0]['generated_text'])

 ChatGPT is an advanced language model developed by Microsoft. It's part of a class of AI models known as "large language models" (LLMs).

ChatGPT is designed to understand and generate human-like text. It can perform a wide range of tasks, such as answering questions, engaging in conversation, summarizing texts, and more.

Here are some key points about ChatGPT:

1. Natural Language Understanding: ChatGPT can understand the context and nuances of human language, enabling it to generate coherent and contextually relevant responses.

2. Versatilian Capabilities: ChatGPT can be used for various applications, such as customer service, content creation, language learning, and more.

3. Continuous Learning: ChatGPT is trained on diverse and extensive datasets, which allows it to learn and adapt to new information and contexts.

4. Ethical Considerations: As with any AI technology, there are ethical considerations to be aware of, such as privacy, bias, and the potential for misuse.

In summa

In [9]:
## Instead of:
# peft_config = LoraConfig(
#     task_type='CAUSAL_LM',
#     r=16,
#     lora_alpha=32,  # thumb rule: lora_alpha should be 2*r
#     bias='none',
#     target_modules='all-linear',
# )
# model = get_peft_model(model, peft_config)
## use:

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,  # rule: lora_alpha should be 2*r
    lora_dropout=0.05,  # Unsloth supports any, but = 0 is optimized
    bias='none',  # Unsloth supports any, but = 'none' is optimized
    # Unsloth does not allow 'all-linear' => manually specify target modules: 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    use_gradient_checkpointing='unsloth',  # True or 'unsloth' for very long context
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [10]:
import os

home_directory = os.path.expanduser("~")
output_dir = os.path.join(home_directory,'output/unsloth-phi-3.5-mini-instruct-guanaco')

training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True, # Gradient checkpointing improves memory efficiency, but slows down training,
        # e.g. Mistral 7B with PEFT using bitsandbytes:
        # - enabled: 11 GB GPU RAM and 8 samples/second
        # - disabled: 40 GB GPU RAM and 12 samples/second
    gradient_checkpointing_kwargs={'use_reentrant': False},  # Use newer implementation that will become the default.
    optim='adamw_torch',
    learning_rate=2e-4,  # QLoRA suggestions: 2e-4 for 7B or 13B, 1e-4 for 33B or 65B
    logging_strategy='steps',  # 'no', 'epoch' or 'steps'
    logging_steps=10,
    save_strategy='no',  # 'no', 'epoch' or 'steps'
    # save_steps=2000,
    # num_train_epochs=5,
    max_steps=100,
    bf16=False,  # mixed precision training
    report_to='none',  # disable wandb
    max_seq_length=1024,
)

In [11]:
def formatting_func(entry):
    return tokenizer.apply_chat_template(entry['messages'], tokenize=False)

In [12]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=guanaco_train,
    processing_class=tokenizer,
    formatting_func=formatting_func,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

train_result = trainer.train()
print("Training result:")
print(train_result)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,846 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/2,039,024,640 (1.47% trained)


Step,Training Loss
10,1.433100
20,1.164400
30,1.213300
40,1.180300
50,1.203100
60,1.287600
70,1.164100
80,1.104000
90,1.127800
100,1.133200


Training result:
TrainOutput(global_step=100, training_loss=1.2010794162750245, metrics={'train_runtime': 295.088, 'train_samples_per_second': 2.711, 'train_steps_per_second': 0.339, 'total_flos': 1.230171564294144e+16, 'train_loss': 1.2010794162750245})


In [14]:
%%time
output_after_training = pipe(guanaco_test[3]['messages'][:1], max_new_tokens=256, return_full_text=False, do_sample=False)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


CPU times: user 9.65 s, sys: 509 μs, total: 9.65 s
Wall time: 9.69 s


In [15]:
print(output_after_training[0]['generated_text'])

 I am Phi, a Phi language model. I am not ChatGPT, but I am a language model that is similar to ChatGPT. I am trained on a large dataset of text, and I can generate text based on the input I receive. I am not sentient or conscious, and I do not have opinions or beliefs. I am a tool that can be used to generate text, and I am here to help you with any questions you may have.                                           


In [ ]:
# Shut down the kernel to release memory
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)

{'status': 'ok', 'restart': False}

: 